<a href="https://colab.research.google.com/github/andygsa/BollyGen/blob/master/Bolly_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#data=pd.read_csv("/content/drive/My Drive/All_Lyrics/Hindi_Songs_DS.csv")

I uploaded the Songs dataset on Drive

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
import numpy as np
import pandas as pd


class OneHotWayEncoding():

    def dataimport(self):
        path = '/content/drive/My Drive/All_Lyrics/Hindi_Songs_DS.csv'
        self.df = pd.read_csv(path)
        print(self.df.head(5))

    def tocorpus(self):
        self.text = self.df['Lyrics'].str.cat(sep='\n').lower()
        # Output the length of the Corpus
        print('corpus length : ', len(self.text))

        # Create a sorted list of the Characters
        self.chars = sorted(list(set(self.text)))
        print('Total Chars : ', len(self.chars))

        # Reducing the length of Corpus
        #self.text = self.text[:900999]
        #print('Truncated Corpus Length : ', len(self.text))

    def featurelabeldataset(self):
        # Create a dictionary where given a character, you can look up the index and vice versa
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

        # cut the text in semi-redundant sequences of maxlen characters
        self.maxlen = 40  # The window size
        self.step = 3  # The steps between the windows
        sentences = []
        next_chars = []

        # Step through the text via 3 characters at a time, taking a sequence of 40 bytes at a time.
        # There will be lots ofo overlap
        for i in range(0, len(self.text) - self.maxlen, self.step):
            sentences.append(self.text[i: i + self.maxlen])  # range from current index i for max length characters
            next_chars.append(self.text[i + self.maxlen])  # the next character after that

        self.sentences = np.array(sentences)
        self.next_chars = np.array(next_chars)
        print('Number of sequences:', len(sentences))

        data = {'Sentences': sentences, 'Next Char': next_chars}

        # Create DataFrame
        newdf = pd.DataFrame(data)

        # Print the output.
        print(newdf)

    def getdata(self,sentences, next_chars):
        X = np.zeros((len(sentences), self.maxlen, len(self.chars)), dtype=np.bool)
        y = np.zeros((len(sentences), len(self.chars)), dtype=np.bool)
        length = len(sentences)
        index = 0
        for i in range(len(sentences)):
            sentence = sentences[i]
            for t, char in enumerate(sentence):
                X[i, t, self.char_indices[char]] = 1
            y[i, self.char_indices[next_chars[i]]] = 1
        return X, y

    def buildmodel(self):
        # build the model: a single LSTM
        print('Build model...')
        self.model = Sequential()
        self.model.add(LSTM(128, input_shape=(self.maxlen, len(self.chars))))
        self.model.add(Dense(len(self.chars)))
        self.model.add(Activation('softmax'))
        self.model.compile(loss='categorical_crossentropy', optimizer='adam')

        print("Compiling model complete...")

    def trainingmodel(self):
        self.dataimport()
        self.tocorpus()
        self.featurelabeldataset()
        self.buildmodel()
        X, y = self.getdata(self.sentences, self.next_chars)
        # The training
        print('Training...')
        # Use this if they all fit into memory
        history = self.model.fit(X, y, batch_size=128, epochs=50, verbose=0)
        # Save the model
        self.model.save('onehot-modelweights.h5')

    def sample(init,preds, temperature=1.0):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)

    def runmodel(self,sentence):
        variance = 0.4
        generated = ''
        original = sentence
        window = sentence
        for i in range(400):
            x = np.zeros((1, self.maxlen, len(self.chars)))
            for t, char in enumerate(window):
                x[0, t, self.char_indices[char]] = 1.

            preds = self.model.predict(x, verbose=0)[0]
            next_index = self.sample(preds, variance)
            next_char = self.indices_char[next_index]

            generated += next_char
            window = window[1:] + next_char

        print(original + generated)

if __name__ == '__main__':
    start = OneHotWayEncoding()
    start.trainingmodel()
    start.runmodel("sapno mein khoya deewana sakshi kaha hai")

   Unnamed: 0                                             Lyrics
0           0  \n\naah ko chaahiye ik umra asar honay tak\nka...
1           1  \n\n aaaa\naeyy ghar ye tera sadaa na mera hai...
2           2  \n\n haay kis but ki muhabbat mein giraftaar h...
3           3  \n\n song idhar phir bhi aana udhar jaane waal...
4           4  \n\n aana meri jan meri jan\nsunday ke sunaday...
corpus length :  10954826
Total Chars :  28
Number of sequences: 3651596
                                          Sentences Next Char
0        \n\naah ko chaahiye ik umra asar honay tak        \n
1         ah ko chaahiye ik umra asar honay tak\nka         u
2         ko chaahiye ik umra asar honay tak\nkaun          j
3         chaahiye ik umra asar honay tak\nkaun jee         t
4         ahiye ik umra asar honay tak\nkaun jeeta          h
...                                             ...       ...
3651591   aan hoon\naapano me bhi khud se main to a         n
3651592    hoon\naapano me bhi khud se ma

In [ ]:
start.runmodel("sapno mein khoya deewana sakshi kaha hai")